In [71]:
from haystack.preprocessor.utils import convert_files_to_dicts
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.file_converter.txt import TextConverter
from haystack.file_converter.pdf import PDFToTextConverter
from haystack.file_converter.docx import DocxToTextConverter
from haystack.preprocessor.preprocessor import PreProcessor
from haystack import Finder
from haystack.document_store.memory import InMemoryDocumentStore
from haystack.retriever.sparse import TfidfRetriever
from haystack.retriever.dense import DensePassageRetriever, EmbeddingRetriever
from ipywidgets import IntProgress
from haystack.pipeline import ExtractiveQAPipeline

In [72]:
all_docs = convert_files_to_dicts(dir_path="./uploaded_disclosure_docs/")

02/20/2021 21:15:17 - INFO - haystack.preprocessor.utils -   Converting uploaded_disclosure_docs/Google_10_Q.pdf
02/20/2021 21:15:18 - INFO - haystack.preprocessor.utils -   Converting uploaded_disclosure_docs/Google_Environmental_Report_2019.pdf
02/20/2021 21:15:18 - INFO - haystack.preprocessor.utils -   Converting uploaded_disclosure_docs/Google_10_K.pdf


In [73]:
all_docs

[{'text': 'UNITED STATES\n\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n________________________________________________________________________________________\n\nFORM 10-Q\n\n(Mark One)\n\n________________________________________________________________________________________\n\n QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\nFor the quarterly period ended September 30, 2020 OR\n TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\nFor the transition period from _______ to _______ Commission file number: 001-37580\n________________________________________________________________________________________\n\nAlphabet Inc.\n\nDelaware\n\n(Exact name of registrant as specified in its charter)\n________________________________________________________________________________________\n\n61-1767919\n\n(State or other jurisdiction of incorporation or organization)\n\n(I.R.S. Employer Identificat

In [74]:
document_store = InMemoryDocumentStore()

In [75]:
document_store.write_documents(all_docs)

In [76]:
retriever = TfidfRetriever(document_store=document_store)


# retriever = DensePassageRetriever(document_store=document_store,
#                                   query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
#                                   passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
#                                   max_seq_len_query=64,
#                                   max_seq_len_passage=256,
#                                   batch_size=16,
#                                   use_gpu=False,
#                                   embed_title=True,
#                                   use_fast_tokenizers=True)

# # Important: 
# # Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# # previously indexed documents and update their embedding representation. 
# # While this can be a time consuming operation (depending on corpus size), it only needs to be done once. 
# # At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
# document_store.update_embeddings(retriever)


# retriever = EmbeddingRetriever(document_store=document_store,
#                                embedding_model="deepset/sentence_bert")

# document_store.update_embeddings(retriever)

02/20/2021 21:15:18 - INFO - haystack.retriever.sparse -   Found 669 candidate paragraphs from 3 docs in DB


In [77]:
reader = FARMReader(model_name_or_path="deepset/xlm-roberta-large-squad2")
# https://huggingface.co/models?pipeline_tag=question-answering

02/20/2021 21:15:18 - INFO - farm.utils -   Using device: CPU 
02/20/2021 21:15:18 - INFO - farm.utils -   Number of GPUs: 0
02/20/2021 21:15:18 - INFO - farm.utils -   Distributed Training: False
02/20/2021 21:15:18 - INFO - farm.utils -   Automatic Mixed Precision: None
02/20/2021 21:15:41 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
02/20/2021 21:15:41 - INFO - farm.utils -   Using device: CPU 
02/20/2021 21:15:41 - INFO - farm.utils -   Number of GPUs: 0
02/20/2021 21:15:41 - INFO - farm.utils -   Distributed Training: False
02/20/2021 21:15:41 - INFO - farm.utils -   Automatic Mixed Precision: None
02/20/2021 21:15:41 - INFO - farm.infer -   Got ya 15 parallel workers to do inference ...
02/20/2021 21:15:41 - INFO - farm.infer -    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 
02/20/2021 21:15:41 - INFO - farm.infer -   /w\  /w\  /w\  /w\  /w\  /w\  /w\  /|\  /w\  /w\  /w\  /w\  /w\  /

In [78]:
pipe = ExtractiveQAPipeline(reader, retriever)

In [79]:
simple_questions = [
    "How does your board oversee climate issues?",
    "Who is responsible for climate-related issues?",
    "How are climate issues monitored?",
    "Do you identify and respond to climate risks and opportunities?",
    "How do you define long-term time horizons?",
    "How do you identify and respond to climate risks and opportunities?",
    "Which climate risk types do you consider?",
    "Do you assess your portfolio's exposure to climate risks and opportunities?",
    "How do you assess your portfolio's exposure to climate risks and opportunities?",
    "Do you request climate-related information from your clients or investees?",
    "Do climate risks have a financial impact on your business?",
    "What climate risks may have a financial impact on your business?",
    "Are there any climate opportunities that have financial impact on your business?",
    "What climate opportunities will have a financial impact on your business?",
    "Have climate risks and opportunities influenced your strategy or finances?",
    "Do you analyze climate scenarios to inform your strategy?",
    "What climate scenarios do you analyze?",
    "How have climate risks and opportunities influenced your strategy?",
    "How have climate risks and opportunities influenced your financial planning?",
    "Do climate issues effect your external asset manager or independent asset manager selection?",
    "How do climate issues effect your external asset manager or independent asset manager selection?",
    "Did you have an emissions target?",
    "What is your absolute emissions target progress?",
    "What is your emissions intensity target progress?",
    "Did you have climate targets?",
    "What are your targets to increase low-carbon energy consumption or production?",
    "What are your methane-reduction targets?",
    "What were your buildings' operational Scope 1 emissions in metric tons of carbon dioxide?",
    "What were market-based power usage Scope 2 emissions in metric tons of carbon dioxide?",
    "What were your employees' Scope 3 emissions in metric tons of carbon dioxide?",
    "What climate metrics do you use?",
    "Do you analyze how your portfolio impacts the climate?",
    "What are your Scope 3 portfolio emissions?",
    "What is your Scope 3 portfolio impact from Category 15 Investments with alternative carbon footprinting or exposure metrics?",
    "Why do you not conduct analysis to understand how your portfolio impacts the climate with Scope 3 portfolio impact from Category 15 Investments with alternative carbon footprinting or exposure metrics?"
]

In [80]:
prediction = pipe.run(query=simple_questions[11], top_k_retriever=5, top_k_reader=5)

In [81]:
print_answers(prediction, details="minimal")

[   {   'answer': 'greenhouse gas emissions',
        'context': ' most significant global challenges of our time, and '
                   'continued greenhouse gas emissions pose an existential '
                   'threat to humanity. The science behind thi'},
    {   'answer': 'extreme weather events, and plastic pollution',
        'context': 'nt challenges of our time, including climate change, '
                   'extreme weather events, and plastic pollution, among '
                   'others.\n'
                   'Climate change is one of the most si'},
    {   'answer': "Google's impact on sustainability, the importance of "
                  'environmental sustainability issues to our business '
                  'strategy, and the perspectives of a diverse range of '
                  'stakeholders outside of Google. The issue areas that were '
                  'found to be of greatest relevance include our data centers, '
                  'workplaces, users, water ste